# SimpleFormat files exploration and processing
La idea es

* observar estructura de la tabla 
* limpiar aquella cosas que no sean utilizadas
* descartar todos aquellos usuarios que no sean de la telco pues no pueden proveer info de su geolocalizacion. 
* Generar un mapping de telco users solamente
* Catalogar las antenas como pertenecientes o no a regiones del *"Gran Chaco"* mejicano

## definiciones utilizadas
* timestamp arranca en 0 segundos para 01/01/2012 00:00am
* el antennaID esta atado solamente al usuario de la TelCo. Los users de la TelCo van todos en la primer columna (independientemente del nombre de la columna..) entonces si la llamada es entrante quiere decir que el user de la telco esta recibiendo un llamado en esa antennaID. si la llamada es saliente es al reves. (Podria pasar que sea comunicaciones inter-Telco y ahi habria dos records uno saliente y otro entrante pero con los mismos 2 userIDs).
* La Direction, viene dada relativa al user de la primer columna (incoming si entra y viceversa si es outgoing).

### el dataset vendria con este header
 {'Target':np.int32 \\\  'Destination':np.uint32 \\\  'TimeStamp':np.uint32 \\\   'Duration':np.uint16 \\\  'AntennaID':np.uint16}

por lo cual seria conveniente eliminar los headers de cada .txt.gz para parsear rapidamente, asignar dtypes y despues volver a asignar column names aca en pandas (u otro program)

In [ ]:
%autosave 180

In [1]:
import pandas as pd; import numpy as np; import os;import random;
pd.set_option('display.max_rows', 300)
from matplotlib import pyplot as plt
%matplotlib inline
import os
import time

In [2]:
np.random.seed(2015)

In [3]:
antennas= pd.read_csv('/home/juan/mobility-study/surrogated_antennas_complete.txt',sep = "|",header=0,index_col=0)
antennas.index.rename("AntennaID",inplace=True) 

In [ ]:
!ls /grandata/simple_format/

In [4]:
#ejemplo  "/grandata/simple_format/01-02012014.txt.gz"
#seteamos el lugar de trabajo
rootdir="/grandata/simple_format/"
os.chdir(rootdir)
year = "2015"; 
month_start= "01";
#todos estos sum_links terminan 2 meses mas tardes
month_end= str(int(month_start)+2)
if int(month_end )<10:
    month_end="0"+month_end
    
day_start = "01"
day_end = str(int(day_start)+1)
if int(day_end )<10:
    day_end="0"+ day_end

input_file= rootdir +"simple_format_{y}{ms}.txt.gz"\
                .format(y=year,ms=month_start,me=month_end,
                       ds=day_start,de=day_end)

In [ ]:
def get_output_file(hash_map=False,night_filter=False,week_end=False):
    output = "/home/juan/mobility-study/output"
    if week_end == True:
        output = output + "_wkend"
    if night_filter == True:
        output = output + "_ngtfilter"
    if hash_map == True:
        output = output + "_user_hash_map"
    return output + ".txt"

In [5]:
!zcat $input_file | head -n10

USER OTHER_USER DIRECTION TIMESTAMP DURATION ANTENNA_ID
216E0E920C0B51093276E270CC36CB72 5492B0033211A161BA4DCECD83A6690E O 94695418 88 1
73C2C1F8233884659CC65DF58BA2D031 0ECA1BB365D1BD1F3B54F5F5225A4337 I 94820900 153 2
73C2C1F8233884659CC65DF58BA2D031 0ECA1BB365D1BD1F3B54F5F5225A4337 O 94842758 189 3
73C2C1F8233884659CC65DF58BA2D031 0ECA1BB365D1BD1F3B54F5F5225A4337 O 94820507 361 2
73C2C1F8233884659CC65DF58BA2D031 0ECA1BB365D1BD1F3B54F5F5225A4337 O 94831395 52 4
73C2C1F8233884659CC65DF58BA2D031 29D24A2CD146FFEDC05757A22BAED052 I 94835642 40 5
73C2C1F8233884659CC65DF58BA2D031 3386587CF389FB2B195C60D92DEB9F81 I 94711785 25 2
73C2C1F8233884659CC65DF58BA2D031 3386587CF389FB2B195C60D92DEB9F81 I 94838044 27 6
73C2C1F8233884659CC65DF58BA2D031 3386587CF389FB2B195C60D92DEB9F81 O 94819123 34 6

gzip: stdout: Broken pipe


### Mappear los user hashes a ints 
La idea es que para esos 5 meses de telco datos que estamos usando, buscamos el conjunto de usuarios de la telco que efectivamente va a ser parte del analisis de mobilidad (sin todavia descartar a los usuarios por filtros de noche o cantidad de uso celular por ejemplo)

Tomamos todo el conjunto de usuarios vistos en los ultimos 5 meses de simple_format y creamos un diccionario que vaya del hash a algun valor en range(1,len(total_num_of_users))

update 06/03: trate de leer todos los files juntos pero a veces se rompen. los leo de a uno, mes a mes

In [72]:
#levanto la tabla de users hash_map.
user_hash_range_map_file = get_output_file(True) + ".gz"
user_hash_range_map = pd.read_csv(user_hash_range_map_file,
                                  sep="|",
                                  index_col=0,
                                  header=0
                )

users = set(user_hash_range_map.index.values)


In [78]:
## script para obtener todos los users en forma de hash.

#ver el tiempo que tarda
start_time = time.time()

if not('users' in globals()):
    users = set()
antennas_ids = set()
#ejemplo  "/grandata/voice/201401_geo/GEO\ Voz\ Pospago\ 01-02012014.txt.gz"
#seteamos el lugar de trabajo
rootdir="/grandata/simple_format/"
os.chdir(rootdir)
year = "2015"; 
months =  ["0"+str(month) for month in range(5,10)]

print("reading {ms}-{y} simple_format file \n \n".format(y=year, ms=month))
month_start= "09";
month_end= str(int(month_start)+2)
if int(month_end )<10:
    month_end="0"+month_end

day_start = "01"
day_end = str(int(day_start)+1)
if int(day_end )<10:
    day_end="0"+ day_end

input_file= rootdir +"simple_format_{y}{ms}.txt.gz"\
                .format(y=year,ms=month_start,me=month_end,
                       ds=day_start,de=day_end)

#el chunk basicamente va leyendo el file de a 'chunksize' cantidad de filas
#subgroup = pd.DataFrame()
#leo de a 40 millones
table = pd.read_csv(
        input_file,
        engine = 'c',
        chunksize = 7*10**7,
#            iterator =True,
        sep = ' ',
        index_col=None,
        header =0,
        #names = ['User', 'Other_User', 'Direction', 'Timestamp'],
        #usecols = ['USER','ANTENNA_ID']
         usecols = ['USER']   
        )

#cuando entramos a este loop, table tiene tantos 'chunks' como el valor entero de la cantidad de lineas en el file
#dividido el tamanyo del chunksize
i=0
for chunk in table:
    i+=1
    if i%3==0:
        print("reading chunk numb {n} for month {m}, time elapsed is {t}".format(n=i,m= month_start,
                                                                                 t=time.time()-start_time))
    #a cada chunk filtro por todos los Targets que tengan resto group modulo passes. y trabajo sobre la tabla
    #subgroup nada mas que ahora tiene pocos usuarios
    #subgroup = subgroup.append(chunk[chunk['Target'] % passes == group])
    users |= set(chunk['USER'].values)
    #antennas_ids |= set(chunk['ANTENNA_ID'].values)
#entonces la idea es que yo ahora solo voy a trabajar, dentro de la tabla

seconds = time.time() - start_time
print("total running time of script is %d " % seconds)

reading 9-2015 simple_format file 
 

reading chunk numb 3 for month 08, time elapsed is 420.625684023
reading chunk numb 6 for month 08, time elapsed is 848.168428898
total running time of script is 867 


### obs1. una corrida de un solo mes tarda aprox 20 mins.
### obs2. cada simpleF file tiene aprox 350m de lineas
### obs3. usamos chunksize de 40m, solo estoy levantando h/ 7gb de ram. Con chunksize 70m levanta h/11gb de ram 
### obs4. hay aprox 5.3m de users 
### obs5. 

In [80]:
len(users)

5257613

In [81]:
#re indexo los usuarios de hash a un mapping de ints
vals = range(1, len(users)+1 ) #con +1 porque me gusta pensar que los datos son positivos y los NAn==-1
#user_hash_range_map = dict(zip(list(users), vals))
users_list = list(users)
user_hash_range_map = pd.DataFrame({'User_hash' : users_list,
                             'User_int' : vals},
                                   #dtype= {'User_int':np.int32}
                                  )

#user_hash_range_map.sort_values(by='User_int',inplace=True)
user_hash_range_map_file = get_output_file(True)
#reindexo
user_hash_range_map.set_index(['User_hash'],inplace=True)
#guardo el file

user_hash_range_map.to_csv(path_or_buf=user_hash_range_map_file,
                           sep = "|"
                        )

#comprimo el archivo

import gzip
import shutil
with open(user_hash_range_map_file, 'rb') as f_in, gzip.open(user_hash_range_map_file+'.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

#elimino el file sin comprimir
!rm $user_hash_range_map_file    

In [85]:
!du -ha $user_hash_range_map_file*

112M	/home/juan/mobility-study/output_user_hash_map.txt.gz


In [87]:
#chequeo que este bien escrito
!zcat $user_hash_range_map_file* | head -n10

User_hash|User_int
32BBE907D578C7B8089413AE28C78999|1
277133E590A0453F0C0657AA573ECC6B|2
778958CAD23103B25C44C26A0B0CEF0C|3
7B299D9DC67285D3EA8526CFB835FFF5|4
5C0E65540627A933C752DA11498BA353|5
028FDC09FE7C3B843112371D84EF7D54|6
62F0FB22E0724D4914AE500E4C979354|7
DFEFFF17DFF3C8A95B42090B273E298C|8
FEB22A2B9AFF2E9489A6383AFC270693|9

gzip: stdout: Broken pipe


## Test
a ver que seria mas rapido si mapear todos los valores desde hash a int y re indexar 
o si tomar un filtro basado en si el ultimo caracter es un numero o una letra.

Result: Claramente es mejor tomar el filtro directamente sobre la tabla y sin reindexar.

In [124]:
import timeit

test_table = pd.read_csv(
        input_file,
        engine = 'c',
#            iterator =True,
        sep = ' ',
        header =0,
        nrows = 3*10**6,
        #names = ['User', 'Other_User', 'Direction', 'Timestamp'],
        #usecols = ['USER','ANTENNA_ID']
         usecols = ['USER']   
        )
test_hash = user_hash_range_map.sample(frac=1/10)

In [113]:
#este es el reindexing a correr.
#user_hash_range_map_dict = user_hash_range_map.to_dict()['User_int']

timeit.timeit('test_table[\'User_int\']= test_table[\'USER\'].map(user_hash_range_map_dict)',
              "from __main__ import test_table, user_hash_range_map_dict",
              number=10)


97.37477016448975

In [ ]:
# filtra en base a una lista de caracteres que uno le pida al hash
#toma como entrada una columna y la devuelve convertida en bool
#test_table2 = test_table['USER'].apply(lambda x: x[-1] in ['1','2','3'] )

timeit.timeit('test_table2 = test_table[\'USER\'].apply(lambda x: x[-1] in [\'1\',\'2\',\'3\'] )',
              "from __main__ import test_table",
              number=10)


## END test

In [ ]:
[l[i:i+n] for i in range(0, len(l), n)]

In [ ]:
#este seria el filtro.
test_table2 = 

In [ ]:
#paso de segundos a horas
    #notar que timestamp arranca en 0 segundos para domingo 01/01/2012 00:00am 
    #con lo cual domingo es el dia 0, lunes el 1, asi..
    
    subgroup['Hour'] =  (subgroup['TimeStamp'].values*1.0/3600)%24
    subgroup['Day']  =  (subgroup['TimeStamp'].values*1.0/(3600*24))%7
    
    #filtro usuarios con pocos o demasiados llamados en general menos de 5 mensuales y mas de 400  
    insignificant_users = subgroup['Target'].value_counts()[(subgroup['Target'].value_counts() < 5) \
                        | (subgroup['Target'].value_counts() > 400)].index.values.tolist()
    subgroup= subgroup.loc[~subgroup['Target'].isin(insignificant_users)]
    
    
    #filtro segun nightfilter y week_end
    if night_filter == True:
        subgroup = subgroup.loc[(subroup['Hour']<8) | (subgroup['Hour']>18)]
    if week_end == True:
        subgroup = subgroup.loc[(subgroup['Day']==0) | (subgroup['Day']==6)]

    
    #   
        
    grouped = subgroup.groupby(['Target', 'AntennaID'])['AntennaID'].agg({'count': np.size})
    grouped.reset_index(inplace=True,drop=False)
    
    del subgroup
    #reordeno dentro de c/ target por el count del antenna, esto me sirve para despues ordenar las antenas por uso
    grouped.sort_values(by=['Target','count'],ascending=False,inplace=True)
    
    ##enriquezco la muestra con datos epidemicos
    #primero agrego a cada antenna del df el dato de si es epidemica
    #despues agrupo por el target y me fijo solo la columna epidemica en c/grupo
    #finalmente sumo en c/ grupo y tomo la parte superior 
#entera de esa division con el largo del grupo. Si uso al menos una antena epidemica entonces esta expuesto(==1) Si no,
# da 0 pues no estuvo expuesto.        
    ##enriquezco la muestra con datos epidemicos

    exposed_info =grouped.join(antennas['EPIDEMIC'], on='AntennaID').\
    groupby('Target')['EPIDEMIC'].\
        agg({'EXPOSED' : lambda x: int( np.ceil(np.sum(x)*1.0/np.size(x)) )}) 
    
    #actualizo la tabla
    grouped = grouped.join(exposed_info['EXPOSED'],on="Target")
    del exposed_info
    
    
    #creo la tabla filtrada solo por users, que es la que voy a terminar guardando (hay tantos rows como users)
    output_table = grouped.drop_duplicates(subset = 'Target', keep='first')
    #re indexo
    output_table.index = output_table['Target'].values
    
    #agrupo ahora la tabla por target para hacer todos los calculos en los grupos
    
    grouped = grouped.groupby('Target')  
    
    #aca voy a ir agregando las top 10 antennas utilizadas por el user, Si no llego a 10 antennas, relleno con NaNs
    for i in range(0,10):
        #me quedo con la i-esima fila de c/grupo (si no hay fila, no toma en cuenta ese Target)
        
        buffer_table = grouped.nth(i)[['AntennaID','count']]
        #renombre a iesima AntennaId e iesimo count
        buffer_table.columns=['AntennaID_%i'%i,'count_%i'%i]
        #agrego esta info como nuevas columnas, dejando lo demas como NaNs
        output_table = pd.concat([output_table, buffer_table], axis=1, join_axes=[output_table.index])
    
    del grouped
    #los primeros AntennaIDs ya no me sirven, idem con el primer
    output_table.drop('AntennaID', axis=1, inplace=True)
    output_table.drop('count', axis=1, inplace=True)
    
    #para los datos faltantes dentro del Top10, relleno con -1s, que vendrian a ser los NaNs
    output_table.fillna(-1,inplace=True)
    
    #ojo aca que en el caso que entren hashes entonces no los va a poder convertir
    output_table = output_table.astype(int,copy=False)
    
    #agrego info de EPIDEMIC, asumiendo a alguien como epidemico si al antenna donde vive (la AntennaID_0) 
    #esta catalogada como EPIDEMIC
    test_table =test_table.join(antennas['EPIDEMIC'], on='AntennaID_0')
    
    
    #print(output_table.columns)
    #print("la tabla es:\n") 
    #print(output_table.head(5))
    #print("\n")
    
    #print(output_table.dytpes)
    #pd.to_numeric(output_table['EXPOSED'])
    
    #for i in range(3,22):
    #    pd.to_numeric(output_table[output_table.columns[i]]) 
    #print(output_table.dytpes)
    
    
    
    
    #aca termino guardando el output final pero solo para esos usuarios % pass ==group
    output_table.to_csv(output_file, index = False, 
                   header = False, mode='a',compression='gzip')


In [ ]:
#numero de usuarios en esta tabla
len(np.unique(tabla2.Target.values)), len(np.unique(tabla.Target.values))

In [ ]:
tabla.shape

In [ ]:
#agarro una muestra de 300k para hacer pruebas
sample=tabla.ix[random.sample(tabla.index, 3*(10**5))]
sample2 = tabla.ix[random.sample(tabla.index, 3*(10**5))]

In [ ]:
#enriquezco el dataset con la info de antennas_mexico.csv
enriched_sample = pd.merge(sample,antennas,left_on="AntennaID",right_index=True)
enriched_sample2 = pd.merge(sample2,antennas,left_on="AntennaID",right_index=True)

In [ ]:
enriched_sample.sort_values(by='TimeStamp',ascending=True,inplace=True)
enriched_sample2.sort_values(by='TimeStamp',ascending=True,inplace=True)

In [ ]:
enriched_sample.head()

## Mobility
La idea es ahora mirar dentro de la base (prototipamos con sample) la lista de antenas utilizadas por cada usuario para obtener la variable dependiente $Y$ que vendria a ser si el tipo viajo o no por la zona endemica (visto como un llamado basicamente desde una antena en la epidemic zone). Despues buscaremos atributos, o variables regresoras que logren predecir ese comportamiento en el pasado pero usando solo los atributos en un timeframe "del futuro". Busco atributos que correlacionen con 'haber viajado por la zona endemica en el pasado'
#### pseudocode:
* tomamos el set de origins/targets de la tabla.
* p/c/user lo mapeamos a un conjunto de antenas que sabemos que el tipo uso en algun CDR. Aca usamos alguna operacion split, apply, combine dondeset donde el key va a ser la columna origin y la data es AntennaID. Ahi extraemos la lista de antenas que el user uso. 
* descartamos del dataset los users "vacios". i.e. aquellos que su conjunto de antenas da vacio. Para esto descartamos todas las filas que no tienen ningun user (origin o target) en la tabla split/apply/combine. Asi obtendremos una lista de usuarios de la telco que viajaron por mexico a partir de sus llamados en ese mes. Despues solo hacer un outer join de todas estas tablas resultantes para cada mes. Asi como tambien un join de estos "sets" de antennas por user. 


In [ ]:
grouped = tabla['AntennaID'].groupby(tabla['Target'])

In [ ]:
#con esto extraigo el mapping user --> antennas used that month
user_antenna_map = sample.groupby('Target')['AntennaID'].apply(lambda x: x.tolist()).to_dict();
user_antenna_map2 = sample2.groupby('Target')['AntennaID'].apply(lambda x: x.tolist()).to_dict();

#esto es 'tipo' un head para dicts
dict(user_antenna_map.items()[0:10]), dict(user_antenna_map2.items()[0:10])

from collections import defaultdict
user_antenna_map_concat = defaultdict(list)

#Veo como concatenar dos "maps" distintos y dejando solo las antenas unicas..
for d in (user_antenna_map, user_antenna_map2): # podemos meter cuantos dicts queramos aca..
    for key, value in d.iteritems():
        #list(set()) elimina repetidos
        user_antenna_map_concat[key] = list(set(user_antenna_map_concat[key] + value))


In [ ]:
#para observar la distribucion de veces que un usuario llama en un mes.
tabla['Target'].value_counts()[(tabla['Target'].value_counts()>5) & (tabla['Target'].value_counts() < 500)].hist(bins=100)

In [ ]:
sample['Target'].value_counts()[(sample['Target'].value_counts()>5) & (sample['Target'].value_counts() < 500)].hist(bins=35)

In [ ]:
#con esto extraigo el mapping user --> antennas used that month
user_antenna = tabla.groupby('Target')['AntennaID'].apply(lambda x: x.tolist()).to_dict();
user_antenna2 = tabla.groupby('Target')['AntennaID'].apply(lambda x: x.tolist()).to_dict();

user_antenna_map_concat = defaultdict(list)

#Veo como concatenar dos "maps" distintos y dejando solo las antenas unicas..
for d in (user_antenna_map, user_antenna_map2): # podemos meter cuantos dicts queramos aca..
    for key, value in d.iteritems():
        #list(set()) elimina repetidos
        user_antenna_map_concat[key] = list(set(user_antenna_map_concat[key] + value))


In [ ]:
import subprocess
subprocess.call(" echo \"Hello World \" ",shell=True)

In [ ]:
os.listdir(os.getcwd())    

In [ ]:
os.chdir(rootdir)
os.walk('.').next()[1]

In [ ]:
os.chdir(rootdir)
print "testdata" in os.listdir(os.getcwd()) 
if "testdata" in os.listdir(os.getcwd()):
    os.system("mkdir testdata")

In [ ]:
os.getcwd()

In [ ]:
os.listdir(os.getcwd())

In [ ]:
!ls